<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Notebooks/01-proceso-analisis-exploratorio/EJERCICIO_2__proceso_analisis_exploratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EJERCICIO 2: Proceso Análisis Exploratorio 
Ejercicio para practicar la introducción al EDA (Exploratory Data Analysis).

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1IT6OVy9YS_sbgHF2oU9D0L--fZiyUfij7NxVhu3cAdg/edit?usp=sharing)
*   [Enlace a notebook con introducción al EDA](https://colab.research.google.com/drive/1B37GMOCjKQ0S8NoRI6lALEW9EZtE0Hit#scrollTo=R53FGLN8jo1L)
*   [Enlace a este notebook en Colab](https://colab.research.google.com/drive/1VMkJls178kiMWqEBRXvFxw47UWMcivfm)




# Objetivo
Eres un inversor privado que está considerando el sp500 como producto. Basándote en el histórico del sp500, resuelve las preguntas que puedes ver a continuación.


Primero, claro, debes importar el dataset del sp500 y las librerías necesarias.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Generamos un objeto para subir el archivo
from google.colab import files
uploaded = files.upload()

Saving sp500.csv to sp500.csv


In [3]:
# Convertimos los bytes a formato tabular
import io
buffer_io = io.BytesIO(uploaded['sp500.csv'])


# Cargamos en un dataframe
df_sp500 = pd.read_csv(buffer_io)
df_sp500.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0


# 01 Pregunta 1
### Cuánto valdría mi inversión ahora si hubiera metido 1 dollar el primer día (1927-12-30)
Nota: ese dollar es equivalente a unos $15.22 de ahora por la inflacción. Ha superado el sp500 la inflacción?

In [4]:
# Variación respecto a ayer
df_sp500['Variacion'] = df_sp500['Close']/df_sp500['Close'].shift(1)

In [11]:
# Solución 1: Producto de la variación diaria
np.array(df_sp500['Variacion'].dropna()).prod()

194.98527412231164

In [12]:
# Solución 2: Variación entre el primer y el último registro
df_sp500.iloc[-1,5]/df_sp500.iloc[0,5]

194.98527412231027

# Pregunta 2
### Qué porcentaje de días el sp500 ha acabado por encima del día valor de cierre del día anterior

In [14]:
# Filtro que determina qué días se ha crecido
filtro_verde = df_sp500['Variacion'] > 1
filtro_verde[:5]

0    False
1     True
2    False
3    False
4     True
Name: Variacion, dtype: bool

In [16]:
# Porcentaje que supone sobre el total
filtro_verde.sum()/len(df_sp500)
# Ok, pues se gana un 52.3% de los días (se pierde bastantes días)

0.5236890622990181

# Pregunta 3
Considerando un año como los ultímos 365 días, es decir, los últimos 250 días de bolsa (cierra findes y festivos), 
### Cuál es la ganancia media anual esperable


In [17]:
# Calculamos primero la variación en los últimos 250 registros para cada registro disponible
df_sp500['Variacion ac ult 250'] = df_sp500['Variacion'].rolling(250).agg(lambda x:x.prod())

In [18]:
# Describe nos da no solo la media, sino también otros estadísticos relevantes
df_sp500.describe()

,Open,High,Low,Close,Adj Close,Volume,Variacion,Variacion ac ult 250
count,23323.000000,23323.000000,23323.000000,23323.000000,23323.000000,2.332300e+04,23322.000000,23073.000000
mean,486.821737,489.690119,483.762812,486.918598,486.918598,7.691668e+08,1.000298,1.076378
std,726.187716,730.112168,721.890344,726.278798,726.278798,1.484864e+09,0.012000,0.200249
min,4.400000,4.400000,4.400000,4.400000,4.400000,0.000000e+00,0.795331,0.294235
25%,23.855001,23.855001,23.855001,23.855001,23.855001,1.280000e+06,0.995466,0.960681
50%,99.500000,100.309998,98.720001,99.500000,99.500000,1.734000e+07,1.000482,1.088715
75%,834.035004,843.744995,822.889984,834.704987,834.704987,5.378550e+08,1.005416,1.195957
max,3564.739990,3588.110107,3535.229980,3580.840088,3580.840088,1.145623e+10,1.166096,2.741573


# Pregunta 4
Mi objetivo es invertir una cantidad en el sp500 y extraer mi dinero en 5 años (de nuevo nos referimos a la idea de año descrita en la pregunta anterior). Pero soy tremendamente gafe e invierto el peor día posible de la historia para conseguir mi objetivo.
### Qué día he invertido? Cuánto he perdido al cabo de 5 años? 

In [19]:
250*5

1250

In [20]:
df_sp500['Variacion ac tras 1250'] = df_sp500['Variacion'].rolling(-1250).agg(lambda x:x.prod())

ValueError: ignored

# 01 Pregunta 1
### Cuál es la ganancia media anual esperable
Nota: la bolsa abre unos 250 días al año.

In [ ]:
# También podemos crear nuestra propia función
df_sp500['Variacion ac ult 250'] = df_sp500['Variacion'].rolling(7).agg(lambda x:x.prod())